In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import shap

Import data prepared in previous steps (only data for 2018 survey, full time employment, and removed outliers) and used for all machine learning models. The target is hourly rate and predictors are economic sector (nace), company size (esize_class), gender, age class, profession (lpk), education. All these are categorical variables. Single numerical variable is experience in years.

In [2]:
data = pd.read_csv('../Data/LT_DU_data_for_ML.csv') 
data.head()

,nace,esize_class,gender,age_class,lpk,education,experience,target
0,C,1_49,M,40-49,p721,G2,13,8.20
1,C,1_49,F,40-49,p334,G2,0,2.51
2,M,50_249,F,40-49,p522,G2,18,2.19
3,M,50_249,F,40-49,p522,G2,12,2.19
4,M,50_249,F,14-29,p522,G2,0,2.19


One fifth of records is kept for testing of models.

In [7]:
y = data['target']
X = data.drop(columns='target')

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=0)

print('Dataset lengths:', 'train', len(y_train), ', test', len(y_test))

Dataset lengths: train 26114 , test 6529


# Model and predictions

## Initial ElasticNet model

Initial ElasticNet model used to determine the need of L1 and L2 regularization and test pipline. Numerical feature is scaled with [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) and categorical features are encoded using [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import ElasticNet

numeric_preprocessor = Pipeline(steps=[("scaler", StandardScaler())])

categorical_preprocessor = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ['nace', 'esize_class', 'gender', 'age_class', 'lpk', 'education']),
        ("numerical", numeric_preprocessor, ['experience'])
    ]
)

model = Pipeline([('prep', preprocessor), ('regr', ElasticNet())])
model.fit(X=X_train, y=y_train)

Objective did not converge. You might want to increase the number of iterations. Duality gap: 40121.12559750184, tolerance: 16.912851508193683


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['nace', 'esize_class',
                                                   'gender', 'age_class', 'lpk',
                                                   'education']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['experience'])])),
                ('regr', ElasticNet(alpha=1e-05, l1_ratio=0))])

Function to print model prediction RMSE and R2

In [45]:
def print_model_rmse_r2(model, X_train, y_train, X_test, y_test):
    """ 
    Function to print model prediction RMSE and R2
    for train and test datasets
        Parameters:
        model - model to evaluate
        X_train - train features
        y_train - train target 
        X_test - test features
        y_test - test target
    """
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred)).round(3)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred)).round(3)

    r2_train = r2_score(y_train, y_train_pred).round(3)
    r2_test = r2_score(y_test, y_test_pred).round(3)

    print('Train: RMSE=', rmse_train, ' R2=', r2_train,
        '\nTest: RMSE=', rmse_test, ' R2=', r2_test)

RMSE and R2 of initial model predicions for train and test datasets.

In [46]:
print_model_rmse_r2(model, X_train, y_train, X_test, y_test)

Train: RMSE= 1.752  R2= 0.526 
Test: RMSE= 1.719  R2= 0.542


ElasticNet with default parameters is not suitable.

## Grid Search CV with ElasticNet

We used GridSearch cross validation to tune model parameters alpha and l1_ratio. The lower boundary for alpha vas set to 0.01 according to recommendation presented in [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html). We used low values of both parameters in GridSearch because initial experiments showed that with high values model did not represent accurately the target.

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
model = Pipeline([('prep', preprocessor), ('regr', ElasticNet())])

grid = {
        'regr__alpha': np.linspace(1e-7,  0.1,  9),
        'regr__l1_ratio' : np.linspace(0,  0.6,  9)
         }
search = GridSearchCV(model, grid, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)
results = search.fit(X=X_train,y=y_train)

results_pd = pd.DataFrame(results.cv_results_)
results_pd.filter(regex='rank|regr|mean_test_score|std_test_score',axis=1).sort_values('rank_test_score').head().T

/home/edvinas/miniconda3/envs/tf211/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32162.091049339717, tolerance: 13.587043325527723
  model = cd_fast.sparse_enet_coordinate_descent(
/home/edvinas/miniconda3/envs/tf211/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 31897.398154779938, tolerance: 13.447193401756708
  model = cd_fast.sparse_enet_coordinate_descent(
/home/edvinas/miniconda3/envs/tf211/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32472.85347297553, tolerance: 13.784672284325318
  model = cd_fast.sparse_enet_coordinate_descent(
/home/edvinas/min

,0,1,2,3,4
param_regr__alpha,0.0,0.0,0.0,0.0,0.0
param_regr__l1_ratio,0.0,0.075,0.15,0.225,0.3
mean_test_score,-1.762076,-1.762076,-1.762076,-1.762076,-1.762076
std_test_score,0.028903,0.028903,0.028903,0.028903,0.028903
rank_test_score,1,2,3,4,5


In [54]:
best_model=results.best_estimator_
best_model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['nace', 'esize_class',
                                                   'gender', 'age_class', 'lpk',
                                                   'education']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['experience'])])),
                ('regr', ElasticNet(alpha=1e-07, l1_ratio=0.0))])

In [55]:
print_model_rmse_r2(best_model, X_train, y_train, X_test, y_test)

Train: RMSE= 1.752  R2= 0.526 
Test: RMSE= 1.719  R2= 0.542


Variability of parameter values is significant for best iterations. Best iterations are with alpha = 0.0 and variable l1_ratio. With alpha = 0 model becomes insensitive to l1_ratio. The ElasticNet [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) states, that alpha = 0 is equivalent to an ordinary least square, thus we will use [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression).

## Linear regression

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

numeric_preprocessor = Pipeline(steps=[("scaler", StandardScaler())])

categorical_preprocessor = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ['nace', 'esize_class', 'gender', 'age_class', 'lpk', 'education']),
        ("numerical", numeric_preprocessor, ['experience'])
    ]
)

model = Pipeline([('prep', preprocessor), ('regr', LinearRegression())])
model.fit(X=X_train, y=y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['nace', 'esize_class',
                                                   'gender', 'age_class', 'lpk',
                                                   'education']),
                                                 ('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['experience'])])),
                ('regr', LinearRegression())])

In [58]:
print_model_rmse_r2(model, X_train, y_train, X_test, y_test)

Train: RMSE= 1.752  R2= 0.526 
Test: RMSE= 1.719  R2= 0.542


LinearRegression model accuracy is the same as ElasticNet with alpha = 0.

# Model (Pipline) Serialization 

In [59]:
import joblib
joblib.dump(best_model, './Models/LM_model.joblib')

['./Models/LM_model.joblib']